# Тестирование микросервиса рекомендаций

In [1]:
import random
import requests
import pandas as pd

In [2]:
recommendations_url = "http://127.0.0.1:8000"
features_store_url = "http://127.0.0.1:8010"
events_store_url = "http://127.0.0.1:8020"
headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}

Загрузим треки для визуализации рекомендаций сервисом

In [61]:
items = pd.read_parquet("items.parquet")
def display_recommended_items(item_ids):

    item_columns_to_use = ["item_id", "name", "album_names", "artist_names", "genre_names"]
    
    items_selected = items.query("item_id in @item_ids")[item_columns_to_use]
    items_selected = items_selected.set_index("item_id").reindex(item_ids)
    items_selected = items_selected.reset_index()
    
    display(items_selected)

#### 1. Для пользователя без персональных рекомендаций

Для пользователя без персональных рекомендаций отображаются рекомендации по умолчанию - популярные треки

In [76]:
# выберем пользователя, которого нет в персональных рекомендациях
personal_recs_users = set(pd.read_parquet("recommendations.parquet")['user_id'])
all_users = set(pd.read_parquet("events.parquet")["user_id"])
cold_users = list(all_users.difference(personal_recs_users))
random_cold_user = random.choice(cold_users)
print(f"Выбранный холодный пользователь: {random_cold_user}")

Выбранный холодный пользователь: 525143


In [77]:
k = 10
params = {"user_id": random_cold_user, 'k': k}

resp = requests.post(recommendations_url + "/recommendations", headers=headers, params=params)
if resp.status_code == 200:
    default_recs = resp.json()
else:
    default_recs = []
    print(f"status code: {resp.status_code}")
print(f"{k} рекомендаций по умолчанию для пользователя {random_cold_user}:\n{default_recs}")

10 рекомендаций по умолчанию для пользователя 525143:
{'recs': [53404, 178529, 33311009, 35505245, 24692821, 795836, 6705392, 37384, 32947997, 51241318]}


Отобразим 10 порекомендованных популярных треков (по умолчанию):

In [78]:
display_recommended_items(default_recs["recs"])

,item_id,name,album_names,artist_names,genre_names
0,53404,Smells Like Teen Spirit,"[Skiing Music, Nirvana, Smells Like Teen Spiri...",[Nirvana],"[rock, alternative, allrock]"
1,178529,Numb,"[00s Rock Anthems, Meteora]",[Linkin Park],"[numetal, metal]"
2,33311009,Believer,"[Biking Songs, Music for working 2020, July 20...",[Imagine Dragons],"[rock, allrock]"
3,35505245,I Got Love,[I Got Love],"[Miyagi & Эндшпиль, Рем Дигга]","[rap, rusrap]"
4,24692821,Way Down We Go,"[A/B, Hits and Stars 2018, DFM Dance 8, Antyra...",[KALEO],[indie]
5,795836,Shape Of My Heart,"[Сентиментальные рок-песни, The Best Of 25 Yea...",[Sting],"[rock, pop, allrock]"
6,6705392,Seven Nation Army,"[Elephant, Pay Close Attention : XL Recordings...",[The White Stripes],[alternative]
7,37384,Zombie,"[90er Hits, Creepy Music, I Am Angry, Hellowee...",[The Cranberries],"[rock, allrock]"
8,32947997,Shape of You,"[÷, Summer Vibes, Pop, Shape of You]",[Ed Sheeran],[pop]
9,51241318,In The End,"[Gaming Bass, Video Gaming Music, In the End, ...","[Tommee Profitt, Fleurie, Mellen Gi]",[rnb]


#### 2. Для пользователя с персональными рекомендациями, но без онлайн-истории,

In [79]:
random_user_for_personal = random.choice(list(personal_recs_users))
k = 10
params = {"user_id": random_user_for_personal, 'k': k}

resp = requests.post(recommendations_url + "/recommendations_offline", headers=headers, params=params)
if resp.status_code == 200:
    offline_recs = resp.json()
else:
    offline_recs = []
    print(f"status code: {resp.status_code}")
    
print(f"{k} персональных рекомендаций для пользователя {random_user_for_personal}:\n{offline_recs}") 

10 персональных рекомендаций для пользователя 457086:
{'recs': [6679078, 51241318, 43942453, 23269133, 46409994, 65765, 60292250, 21133234, 65062257, 10270285]}


In [80]:
display_recommended_items(offline_recs["recs"])

,item_id,name,album_names,artist_names,genre_names
0,6679078,Intro,[xx],[The xx],[indie]
1,51241318,In The End,"[Gaming Bass, Video Gaming Music, In the End, ...","[Tommee Profitt, Fleurie, Mellen Gi]",[rnb]
2,43942453,NBA,[АРГУМЕНТЫ],[RSAC],"[local-indie, indie]"
3,23269133,Take Me To The River,[Think],[Kaleida],[electronics]
4,46409994,When I'm Small,"[One Night Off, Eyelid Movies]",[Phantogram],[indie]
5,65765,This Is The Life,"[Workflow Tunes, Young Folks, Tik Tock Hits 20...",[Amy Macdonald],[pop]
6,60292250,Blinding Lights,"[The Highlights, After Hours, Blinding Lights]",[The Weeknd],[rnb]
7,21133234,Midnight City,"[Hurry up, We're Dreaming, Hurry Up, We're Dre...",[M83],[electronics]
8,65062257,Кометы,"[Сказки лесной нимфы, Кометы]",[polnalyubvi],"[local-indie, indie]"
9,10270285,Young And Beautiful,"[Summer Heartbreak, Morning Vibes, Music From ...",[Lana Del Rey],[pop]


#### 3. Для пользователя с персональными рекомендациями и онлайн-историей.

In [81]:
# проверяем что у пользователя нет онлайн истории

k = 10
params = {"user_id": random_user_for_personal, "k": k}
resp = requests.post(events_store_url + "/get", headers=headers, params=params)
if resp.status_code == 200:
    result = resp.json()
else:
    result = None
    print(f"status code: {resp.status_code}")
    
print(f"Последние {k} событий пользователя {random_user_for_personal}: {result}")

Последние 10 событий пользователя 457086: {'events': []}


Выберем треки, которые "прослушает" пользователь:

In [54]:
display(items[items["name"] == 'Цвет настроения чёрный'])
display(items[(items["name"] == 'Капкан') & (items["album_names"].apply(lambda x: 'Наизнанку' in x))])
display(items[(items["name"] == '99 Problems') & (items["artist_names"].apply(lambda x: 'kizaru' in x))])
display(items[(items["name"] == 'Ресницы') & (items["album_names"].apply(lambda x: 'Братья Грим' in x))])

,item_id,name,album_names,artist_names,genre_names
617480,43202169,Цвет настроения чёрный,[Цвет настроения чёрный],"[Филипп Киркоров, ЕГОР КРИД]","[pop, ruspop]"


,item_id,name,album_names,artist_names,genre_names
396562,27077792,Капкан,[Наизнанку],[MOT],"[rap, rusrap]"


,item_id,name,album_names,artist_names,genre_names
963499,93556487,99 Problems,[BANDANA I],"[kizaru, Big Baby Tape]","[rap, rusrap]"


,item_id,name,album_names,artist_names,genre_names
18496,109302,Ресницы,[Братья Грим],[Братья Грим],"[allrock, rusrock]"


In [82]:
# добавляем события взаимодействия пользователя

event_item_ids =  [43202169, 27077792, 93556487, 109302]

for event_item_id in event_item_ids:
    resp = requests.post(events_store_url + "/put", 
                        headers=headers, 
                        params={"user_id": random_user_for_personal, "item_id": event_item_id})
print(f"Добавлены прослушивания пользователем №{random_user_for_personal} треков {event_item_ids}")

Добавлены прослушивания пользователем №457086 треков [43202169, 27077792, 93556487, 109302]


Проверим, что они сохранены в events_store

In [83]:
k = 4
params = {"user_id": random_user_for_personal, "k": k}
resp = requests.post(events_store_url + "/get", headers=headers, params=params)
if resp.status_code == 200:
    result = resp.json()
else:
    result = None
    print(f"status code: {resp.status_code}")
    
print(f"Последние {k} событий пользователя {random_user_for_personal}: {result}")

Последние 4 событий пользователя 457086: {'events': [109302, 93556487, 27077792, 43202169]}


Посмотрим на онлайн рекомендации для пользователя с прослушанными треками русского рэпа и песней 2005 рок-группы

In [84]:
params = {"user_id": random_user_for_personal, 'k': 3} # k - количество подходящих на одно взаимодействие
resp = requests.post(recommendations_url + "/recommendations_online", headers=headers, params=params)
online_recs = resp.json()["recs"]
display_recommended_items(online_recs)

,item_id,name,album_names,artist_names,genre_names
0,93556493,Million,[BANDANA I],"[kizaru, Big Baby Tape]","[rap, rusrap]"
1,93556497,Bandana,[BANDANA I],"[kizaru, Big Baby Tape]","[rap, rusrap]"
2,39917908,Под фонарём,[Под фонарём],[RASA],"[pop, ruspop]"
3,99695439,Сумасшедший,"[Сумасшедший, Sumashedshiy]","[kostromin, EVASHA]","[foreignrap, rap, rusrap]"
4,109310,Кустурица,[Братья Грим],[Братья Грим],"[allrock, rusrock]"
5,105734,Прасковья,[В городе N],[Uma2rman],"[allrock, rusrock]"
6,29212855,На дне,[92 дня],[MOT],"[rap, rusrap]"
7,28193017,Абсолютно всё,[Наизнанку],[MOT],"[rap, rusrap]"
8,41273824,Гучи,[Гучи],[Тимати],"[rap, rusrap]"
9,41028870,Чика,"[Summer Music 2018, Чика]",[Артур Пирожков],"[pop, ruspop]"


Отличный результат! Полное совпадение по жанрам, присутствуют треки из тех же альбомов, что и те, которые были "прослушаны", артисты преимущественно совпадают с тем, кого слушал пользователь. Особенно хорошо выглядит то, что на прослушанный рок-трек Ресницы, сервис порекомендовал самый популярный трек исполнителя этой песни Братья Грим - Кустурица 

Запросим у сервиса для пользователя смешанные онлайн и оффлайн (чередуются онлайн, затем оффлайн) рекомендации

In [85]:
k = 10
params = {"user_id": random_user_for_personal, 'k': k}

resp = requests.post(recommendations_url + "/recommendations", headers=headers, params=params)
if resp.status_code == 200:
    recs = resp.json()
else:
    recs = []
    print(f"status code: {resp.status_code}")
    
print(f"Смешанные оффлайн и онлайн рекомендации для пользователя {random_user_for_personal}: {recs}")

Смешанные оффлайн и онлайн рекомендации для пользователя 457086: {'recs': [93556493, 6679078, 93556497, 51241318, 39917908, 43942453, 99695439, 23269133, 96795310, 46409994]}


Проверим, что было рекомендовано пользователю

In [86]:
display_recommended_items(recs["recs"])

,item_id,name,album_names,artist_names,genre_names
0,93556493,Million,[BANDANA I],"[kizaru, Big Baby Tape]","[rap, rusrap]"
1,6679078,Intro,[xx],[The xx],[indie]
2,93556497,Bandana,[BANDANA I],"[kizaru, Big Baby Tape]","[rap, rusrap]"
3,51241318,In The End,"[Gaming Bass, Video Gaming Music, In the End, ...","[Tommee Profitt, Fleurie, Mellen Gi]",[rnb]
4,39917908,Под фонарём,[Под фонарём],[RASA],"[pop, ruspop]"
5,43942453,NBA,[АРГУМЕНТЫ],[RSAC],"[local-indie, indie]"
6,99695439,Сумасшедший,"[Сумасшедший, Sumashedshiy]","[kostromin, EVASHA]","[foreignrap, rap, rusrap]"
7,23269133,Take Me To The River,[Think],[Kaleida],[electronics]
8,96795310,Домой,[Домой],[MORGENSHTERN],"[rap, rusrap]"
9,46409994,When I'm Small,"[One Night Off, Eyelid Movies]",[Phantogram],[indie]
